In [1]:
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import *

Passengers : dictionnaire extrait du fichier excel

Les sièges sont comptés comme une liste et pas comme une matrice

Sij : var. binaire qui à un passager i associe le siège j

In [ ]:
ranks = 29 #nombre de rangées dans l'avion
n = 81 #nombre de passagers

def modele_statique():
    m=Model('statique')
    S = {(i,j) : m.addVar(vtype = GRB.BINARY, name = f'j{i}') for i in range(1, n+1) for j in range (1, 6*ranks+1)}

    return m